# Yorkshire Water -vedenkulutuksen k-means-mallinnus

In [5]:
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

Ladataan alkuperäinen data käsittelyyn.

In [11]:
db_filepath = Path.cwd().parent / "data" / "Daily m3 1315.csv"
# df_filepath.exists()

# d_df = pd.read_csv("Data/Daily_m3_1315.csv")
d_df = pd.read_csv(db_filepath)

# 24/03/2012 is Saturday
#d_df.index
print(f"Number of columns: {len(d_df.columns)}")
print(f"Number of rows: {len(d_df)}")

Number of columns: 1128
Number of rows: 2596


Etsitään datasta kunnollinen osajoukko, jossa ei ole puuttuvia arvoja.

In [12]:
def etsi_kunnon_df(num_of_properties):
    alku = 3
    loppu = len(d_df.columns) - 1

    kunnon_df = None
    while alku < loppu:
        for a in range(alku, (loppu- alku) // 2, 2):
            sarakkeet = [0, 1, 2] + list(range(a, loppu - a + 1))
            testi_df = d_df.iloc[:, sarakkeet].dropna()
            if len(testi_df) >= num_of_properties:
                return testi_df
        alku += 5
        loppu -= 5
    return kunnon_df

# etsi_kunnon_df(267)
kunnon_df = etsi_kunnon_df(270)

Tarkastetaan negatiiviset arvot, ja asetetaan ne nollaksi.

In [13]:
# negatiiviset arvot kurkataan
arvot = kunnon_df.iloc[:, 3:].to_numpy()

print(arvot[arvot < 0])
print(np.where(arvot < 0))
print(kunnon_df.index[354])
print(kunnon_df.iloc[354, 473])
# asetetaan nollaksi
kunnon_df.iloc[354, 473] = 0

[-0.001]
(array([354], dtype=int64), array([470], dtype=int64))
1526
-0.001


Klusteroidaan data k-means-menetelmän avulla.

In [14]:
kulutusdata = kunnon_df.iloc[:, 3:]

def klusteroi(klusterien_lkm, rng=1337, iter_lkm=420):
    km = KMeans(n_clusters=klusterien_lkm, n_init="auto", max_iter=iter_lkm, random_state=rng)
    km.fit(kulutusdata)

    luokittelu = pd.DataFrame(km.predict(kulutusdata), columns=["luokka"])
    luokittelu.index = kulutusdata.index
    luokittelu["keskikulutus"] = kulutusdata.mean(axis=1)

    lkm = []
    maksimikulutus = []
    minimikulutus = []
    for l in range(klusterien_lkm):
        kulutus = luokittelu[luokittelu["luokka"] == l]["keskikulutus"]
        lkm.append(kulutus.count())
        maksimikulutus.append(kulutus.max())
        minimikulutus.append(kulutus.min())

    esitys_df = pd.DataFrame(range(klusterien_lkm), columns=["luokka"])
    esitys_df["count"] = lkm
    esitys_df["minimikeskikulutus"] = minimikulutus
    esitys_df["maksimikeskikulutus"] = maksimikulutus

    return esitys_df

In [15]:
luokat = klusteroi(8)
luokat.sort_values(by="minimikeskikulutus")

,luokka,count,minimikeskikulutus,maksimikeskikulutus
4,4,363,0.000000,0.189097
0,0,319,0.189561,0.323966
7,7,224,0.321420,0.491225
5,5,83,0.460851,0.742849
2,2,23,0.712334,1.362496
3,3,1,1.439133,1.439133
6,6,2,1.957559,2.251086
1,1,1,11.144535,11.144535
